# Regression Week 2: Multiple Regression (Interpretation)

The goal of this first notebook is to explore multiple regression and feature engineering with existing graphlab functions.

In this notebook you will use data on house sales in King County to predict prices using multiple regression. You will:
* Use SFrames to do some feature engineering
* Use built-in graphlab functions to compute the regression weights (coefficients/parameters)
* Given the regression weights, predictors and outcome write a function to compute the Residual Sum of Squares
* Look at coefficients and interpret their meanings
* Evaluate multiple models via RSS

# Fire up graphlab create

In [1]:
#import graphlab
import pandas as pd
from pandas import read_csv
import numpy as np
from sklearn.linear_model import LinearRegression

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
#sales = graphlab.SFrame('kc_house_data.gl/')

# Split data into training and testing.
We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [3]:
#train_data,test_data = sales.random_split(.8,seed=0)

In [4]:
dtype_dict = {'bathrooms':float, 'waterfront':int, 'sqft_above':int, 'sqft_living15':float, 'grade':int, 'yr_renovated':int, 
              'price':float, 'bedrooms':float, 'zipcode':str, 'long':float, 'sqft_lot15':float, 'sqft_living':float, 
              'floors':str, 'condition':int, 'lat':float, 'date':str, 'sqft_basement':int, 'yr_built':int, 'id':str, 
              'sqft_lot':int, 'view':int}

train_file_name = r'data/kc_house_train_data.csv'
train_data = pd.read_csv(train_file_name, dtype=dtype_dict)

test_file_name = r'data/kc_house_test_data.csv'
test_data = pd.read_csv(test_file_name, dtype=dtype_dict)

In [5]:
test_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,0114101516,20140528T000000,310000.0,3.0,1.0,1430.0,19901,1.5,0,0,...,7,1430,0,1927,0,98028,47.7558,-122.229,1780.0,12697.0
1,9297300055,20150124T000000,650000.0,4.0,3.0,2950.0,5000,2,0,3,...,9,1980,970,1979,0,98126,47.5714,-122.375,2140.0,4000.0
2,1202000200,20141103T000000,233000.0,3.0,2.0,1710.0,4697,1.5,0,0,...,6,1710,0,1941,0,98002,47.3048,-122.218,1030.0,4705.0
3,8562750320,20141110T000000,580500.0,3.0,2.5,2320.0,3980,2,0,0,...,8,2320,0,2003,0,98027,47.5391,-122.070,2580.0,3980.0
4,7589200193,20141110T000000,535000.0,3.0,1.0,1090.0,3000,1.5,0,0,...,8,1090,0,1929,0,98117,47.6889,-122.375,1570.0,5080.0


# Learning a multiple regression model

Recall we can use the following code to learn a multiple regression model predicting 'price' based on the following features:
example_features = ['sqft_living', 'bedrooms', 'bathrooms'] on training data with the following code:

(Aside: We set validation_set = None to ensure that the results are always the same)

In [6]:
example_features = ['sqft_living', 'bedrooms', 'bathrooms']
#example_model = graphlab.linear_regression.create(train_data, target = 'price', features = example_features, 
#                                                  validation_set = None)

In [7]:
#train_data[['sqft_living', 'bedrooms', 'bathrooms']]

In [8]:
example_model = LinearRegression()
example_model.fit(train_data[['sqft_living', 'bedrooms', 'bathrooms']], train_data[['price']])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [9]:
#http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html
print('Coefficients: \n', example_model.coef_)

('Coefficients: \n', array([[   315.40669062, -65081.88711588,   6942.16598637]]))


Now that we have fitted the model we can extract the regression weights (coefficients) as an SFrame as follows:

In [10]:
# example_weight_summary = example_model.get("coefficients")
# print example_weight_summary

# Making Predictions

In the gradient descent notebook we use numpy to do our regression. In this book we will use existing graphlab create functions to analyze multiple regressions. 

Recall that once a model is built we can use the .predict() function to find the predicted values for data we pass. For example using the example model above:

In [11]:
#example_predictions = example_model.predict(train_data)
#print example_predictions[0] # should be 271789.505878

In [12]:
example_predictions = example_model.predict(train_data[['sqft_living', 'bedrooms', 'bathrooms']])
print example_predictions[0] # should be 271789.505878

[ 271789.26537997]


# Compute RSS

Now that we can make predictions given the model, let's write a function to compute the RSS of the model. Complete the function below to calculate RSS given the model, data, and the outcome.

In [13]:
def get_residual_sum_of_squares(model, data, outcome):
    # First get the predictions
    predictions = model.predict(data)
    # Then compute the residuals/errors
    RSS = 0
    for i in range(len(predictions)):
        RSS += ((predictions[i] - outcome[i]) **2)
    
    #residues = model.residues_
    # Then square and add them up
    #RSS = (np.multiply(residues, residues)).sum()
    return(RSS)    

Test your function by computing the RSS on TEST data for the example model:

In [14]:
rss_example_train = get_residual_sum_of_squares(example_model, test_data[['sqft_living', 'bedrooms', 'bathrooms']], 
                                                test_data['price'])
print rss_example_train # should be 2.7376153833e+14

[  2.73761941e+14]


# Create some new features

Although we often think of multiple regression as including multiple different features (e.g. # of bedrooms, squarefeet, and # of bathrooms) but we can also consider transformations of existing features e.g. the log of the squarefeet or even "interaction" features such as the product of bedrooms and bathrooms.

You will use the logarithm function to create a new feature. so first you should import it from the math library.

In [15]:
from math import log

Next create the following 4 new features as column in both TEST and TRAIN data:
* bedrooms_squared = bedrooms\*bedrooms
* bed_bath_rooms = bedrooms\*bathrooms
* log_sqft_living = log(sqft_living)
* lat_plus_long = lat + long 
As an example here's the first one:

In [16]:
train_data['bedrooms_squared'] = train_data['bedrooms'].apply(lambda x: x**2)
test_data['bedrooms_squared'] = test_data['bedrooms'].apply(lambda x: x**2)

In [17]:
# create the remaining 3 features in both TEST and TRAIN data
#np.add(intercept,result_input_result)

train_data['bed_bath_rooms'] = np.add(train_data['bedrooms'], train_data['bathrooms'])
test_data['bed_bath_rooms'] = np.add(test_data['bedrooms'], test_data['bathrooms'])

#HAVE TO CHECK THE LOG
train_data['log_sqft_living'] = train_data['sqft_living'].apply(lambda x: log(x))
test_data['log_sqft_living'] = test_data['sqft_living'].apply(lambda x: log(x))

train_data['lat_plus_long'] = np.add(train_data['lat'], train_data['long'])
test_data['lat_plus_long'] = np.add(test_data['lat'], test_data['long'])

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this feature will mostly affect houses with many bedrooms.
* bedrooms times bathrooms gives what's called an "interaction" feature. It is large when *both* of them are large.
* Taking the log of squarefeet has the effect of bringing large values closer together and spreading out small values.
* Adding latitude to longitude is totally non-sensical but we will do it anyway (you'll see why)

In [18]:
train_data.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bedrooms_squared,bed_bath_rooms,log_sqft_living,lat_plus_long
0,7129300520,20141013T000000,221900.0,3.0,1.00,1180.0,5650,1,0,0,...,0,98178,47.5112,-122.257,1340.0,5650.0,9.0,4.00,7.073270,-74.7458
1,6414100192,20141209T000000,538000.0,3.0,2.25,2570.0,7242,2,0,0,...,1991,98125,47.7210,-122.319,1690.0,7639.0,9.0,5.25,7.851661,-74.5980
2,5631500400,20150225T000000,180000.0,2.0,1.00,770.0,10000,1,0,0,...,0,98028,47.7379,-122.233,2720.0,8062.0,4.0,3.00,6.646391,-74.4951
3,2487200875,20141209T000000,604000.0,4.0,3.00,1960.0,5000,1,0,0,...,0,98136,47.5208,-122.393,1360.0,5000.0,16.0,7.00,7.580700,-74.8722
4,1954400510,20150218T000000,510000.0,3.0,2.00,1680.0,8080,1,0,0,...,0,98074,47.6168,-122.045,1800.0,7503.0,9.0,5.00,7.426549,-74.4282


In [19]:
log(1180.0)

7.07326971745971

**Quiz Question: What is the mean (arithmetic average) value of your 4 new features on TEST data? (round to 2 digits)**

In [25]:
print test_data['bedrooms_squared'].mean()
print test_data['bed_bath_rooms'].mean()
print test_data['log_sqft_living'].mean()
print test_data['lat_plus_long'].mean()

12.4466777016
5.49048238354
7.55027467965
-74.653333554


# Learning Multiple Models

Now we will learn the weights for three (nested) models for predicting house prices. The first model will have the fewest features the second model will add one more feature and the third will add a few more:
* Model 1: squarefeet, # bedrooms, # bathrooms, latitude & longitude
* Model 2: add bedrooms\*bathrooms
* Model 3: Add log squarefeet, bedrooms squared, and the (nonsensical) latitude + longitude

In [20]:
model_1_features = ['sqft_living', 'bedrooms', 'bathrooms', 'lat', 'long']
model_2_features = model_1_features + ['bed_bath_rooms']
model_3_features = model_2_features + ['bedrooms_squared', 'log_sqft_living', 'lat_plus_long']

Now that you have the features, learn the weights for the three different models for predicting target = 'price' using graphlab.linear_regression.create() and look at the value of the weights/coefficients:

In [26]:
# Learn the three models: (don't forget to set validation_set = None)
model_1 = LinearRegression()
model_1.fit(train_data[model_1_features], train_data[['price']])
print('Coefficients: \n', model_1.coef_)

('Coefficients: \n', array([[  3.12258646e+02,  -5.95865332e+04,   1.57067421e+04,
          6.58619264e+05,  -3.09374351e+05]]))


In [27]:
model_2 = LinearRegression()
model_2.fit(train_data[model_2_features], train_data[['price']])
print('Coefficients: \n', model_2.coef_)

('Coefficients: \n', array([[  3.12258646e+02,  -4.49599361e+04,   3.03333391e+04,
          6.58619264e+05,  -3.09374351e+05,  -1.46265970e+04]]))


In [28]:
model_3 = LinearRegression()
model_3.fit(train_data[model_3_features], train_data[['price']])
print('Coefficients: \n', model_3.coef_)

('Coefficients: \n', array([[  5.17895745e+02,   1.10062820e+04,   1.27309278e+04,
          5.34373572e+05,  -4.06758960e+05,   2.37372098e+04,
         -9.44734086e+03,  -5.36751710e+05,   1.27614612e+05]]))


In [22]:
# Examine/extract each model's coefficients:
#print('Coefficients: \n', model_1.coef_)

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 1?** +

**Quiz Question: What is the sign (positive or negative) for the coefficient/weight for 'bathrooms' in model 2?** +

Think about what this means.

# Comparing multiple models

Now that you've learned three models and extracted the model weights we want to evaluate which model is best.

First use your functions from earlier to compute the RSS on TRAINING Data for each of the three models.

In [37]:
model_1_rss_example_train = get_residual_sum_of_squares(model_1, train_data[model_1_features], 
                                                train_data['price'])
print model_1_rss_example_train 

[  9.67879963e+14]


In [38]:
model_2_rss_example_train = get_residual_sum_of_squares(model_2, train_data[model_2_features], 
                                                train_data['price'])
print model_2_rss_example_train 

[  9.67879963e+14]


In [39]:
model_3_rss_example_train = get_residual_sum_of_squares(model_3, train_data[model_3_features], 
                                                train_data['price'])
print model_3_rss_example_train

[  9.03904707e+14]


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TRAINING Data?** Is this what you expected?

Now compute the RSS on on TEST data for each of the three models.

In [40]:
model_1_rss_example_test = get_residual_sum_of_squares(model_1, test_data[model_1_features], 
                                                test_data['price'])
print model_1_rss_example_test 

[  2.25500470e+14]


In [41]:
model_2_rss_example_test = get_residual_sum_of_squares(model_2, test_data[model_2_features], 
                                                test_data['price'])
print model_2_rss_example_test 

[  2.25500470e+14]


In [42]:
model_3_rss_example_test = get_residual_sum_of_squares(model_3, test_data[model_3_features], 
                                                test_data['price'])
print model_3_rss_example_test 

[  2.97935497e+14]


**Quiz Question: Which model (1, 2 or 3) has lowest RSS on TESTING Data?** Is this what you expected? Think about the features that were added to each model from the previous.

In [36]:
#2